# Determine support assignment gaps

In [ ]:
import pandas as pd
%matplotlib inline

from entities import CoachingPracticeFinance, ExpenseLineItem, HoursLineItem

practice = CoachingPracticeFinance.load("../practice.json")

In [ ]:
period_start = pd.to_datetime("2022-11-01")
period_end = pd.to_datetime("2023-10-31")
report_date_range = pd.bdate_range(period_start, period_end, freq="C", holidays=practice.statutory_holiday_list)
report_frame = pd.DataFrame(report_date_range, columns=["date"])
report_frame

In [ ]:
assignment_coverage = []
columns = ['date'] + [a.support_area_code for a in practice.area_assignments]
for date in report_date_range:
    row = [date]
    for a in practice.area_assignments:
        start = pd.to_datetime(a.start_date)
        end = pd.to_datetime(a.end_date)
        row.append(start <= date and end >= date)
    assignment_coverage.append(row)
assignments_frame = pd.DataFrame(assignment_coverage, columns=columns)
assignments_frame

In [ ]:
df = report_frame.merge(assignments_frame, how="outer", on="date")
df

In [ ]:
for a in practice.area_assignments:
    df[f"{a.support_area_code}_group"] = (df[a.support_area_code].diff() != 0).cumsum()
df

In [ ]:
from tabulate import tabulate

lines = []
for a in practice.area_assignments:
    unstaffed = df[df[a.support_area_code] == False]
    report = unstaffed.groupby(f"{a.support_area_code}_group")['date'].agg(['min', 'max'])
    report = report.reset_index(drop=True)
    if report.count()[0] > 0:
        lines.append("")
        lines.append(f"{a.support_area_code} is Unstaffed")
        lines.append("")
        report['min'] = report['min'].dt.strftime('%Y-%m-%d')
        report['max'] = report['max'].dt.strftime('%Y-%m-%d')
        report.columns = ['Start Date', 'End Date']
        lines.append(tabulate(report, showindex=False))
print("\n".join(lines))